In [1]:
import pandas as pd
import numpy as np
import datetime 
from datetime import datetime 
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

### Functions needed

In [2]:
def read_dataframe(link):
    df = pd.read_parquet(link)
    
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [3]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df_validate = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')

# Q1. Downloading the data


In [4]:
_ = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [5]:
_.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


### Read the data for January. How many columns are there?


In [6]:
print(f'The number of columns = {_.shape[1]}')

The number of columns = 19


# Q2. Computing duration

In [7]:
import datetime 
from datetime import datetime 

In [8]:
_['duration'] = (_['tpep_dropoff_datetime'] - _['tpep_pickup_datetime']).dt.total_seconds() / 60

_['duration'].std()

34.851053592192876

# Q3. Dropping outliers

In [9]:
# Calculate the fraction of records left after removing outliers
fraction_left = len(df_train) / len(_)

# Percentage 
print(f"Fraction of records left after dropping outliers (rounded): {round(fraction_left * 100)} %")


Fraction of records left after dropping outliers (rounded): 98 %


# Q4. One-hot encoding

In [10]:
len(df_train), len(df_validate)

(2898906, 2938060)

In [11]:
# df_train['PU-DO'] = df_train['PULocationID'] + '-' + df_train['DOLocationID']
# df_validate['PU-DO'] = df_validate['PULocationID'] + '-' + df_validate['DOLocationID']

In [12]:
# categorical = ['PU-DO']
# numerical = ['trip_distance']

# dv = DictVectorizer(sparse = True)

# train_dicts = df_train[categorical + numerical].to_dict(orient='records')
# X_train = dv.fit_transform(train_dicts)

# validate_dicts = df_validate[categorical + numerical].to_dict(orient='records')
# X_validate = dv.fit_transform(validate_dicts)

In [13]:
# X_train.shape

In [14]:
categorical = ['PULocationID', 'DOLocationID']

In [15]:
# Convert DataFrame to a list of dictionaries
train_dict = df_train[categorical].to_dict(orient='records')

In [16]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse = True)

In [17]:
dv.fit(train_dict)
X_train=dv.transform(train_dict)
X_train.shape

(2898906, 518)

In [18]:
# Get the dimensionality of the feature matrix
num_columns = X_train.shape[1]

print("Dimensionality of the feature matrix:", num_columns)

Dimensionality of the feature matrix: 518


# Q5. Training a model


In [19]:
target = 'duration'
y_train = df_train[target].values
y_validate = df_validate[target].values

In [20]:
# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)  

# Predict on the training data
predictions_train = model.predict(X_train)

# Calculate the RMSE on the training data
rmse_train = mean_squared_error(y_train, predictions_train, squared=False)

print("RMSE on train:", rmse_train)

# Save the trained model using pickle
with open('linear_regression_model.pkl', 'wb') as f:
    pickle.dump(model, f)

RMSE on train: 7.946173359562653


# Q6. Evaluating the model


In [21]:
# Convert DataFrame to a list of dictionaries
validate_dict = df_validate[categorical].to_dict(orient='records')

from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse = True)

dv.fit(validate_dict)
X_validate=dv.transform(validate_dict)
X_validate.shape

# Get the dimensionality of the feature matrix
num_columns = X_validate.shape[1]

print("Dimensionality of the feature matrix:", num_columns)

Dimensionality of the feature matrix: 514


In [22]:
# Load the saved model using pickle
with open('linear_regression_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Predict on the validation data
predictions_validation = loaded_model.predict(X_validate)

# Calculate the RMSE on the validation data
rmse_validation = mean_squared_error(y_validate, predictions_validation, squared=False)

print("RMSE on validation:", rmse_validation)

ValueError: X has 514 features, but LinearRegression is expecting 518 features as input.